Lendo os dados e gravando no Dataframe

In [1]:
import sys
import os

# Adiciona o caminho da pasta 'scripts' ao sys.path
sys.path.append(os.path.abspath('../scripts'))

In [2]:
# Importação das bibliotecas
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import time
import pandas as pd
from db import *

engine = get_postgres_engine()


Utilizando script SQL para fazer o agrupamento e consolidar a base GOLD

In [29]:
# Lê a tabela diretamente para um DataFrame
df = pd.read_sql("""SELECT "NomeProduto", 
"CategoriaProduto",  
SUBSTRING("DataVenda_Formatada", 1,10) AS "DataVenda", 
"Estado" AS "EstadoFilial",
SUM("PreçoProduto"::numeric) AS "ValorTotalVenda"
FROM public."TB_VENDAS_EMPRESA_B_SILVER"
GROUP BY 1,2,3,4""", engine)
# Exibe os primeiros registros
print(df.head())

         NomeProduto  CategoriaProduto   DataVenda EstadoFilial  \
0        Pó compacto            Roupas  11/12/2022        Bahia   
1  Máscara de cílios  Eletrodomésticos  18/02/2023     Amazonas   
2          Corretivo         Alimentos  01/07/2023        Bahia   
3      Batom líquido       Eletrônicos  18/08/2023        Ceara   
4        Batom matte       Eletrônicos  25/02/2023        Ceara   

   ValorTotalVenda  
0            58.69  
1            72.50  
2             7.61  
3            78.87  
4            54.23  


In [ ]:
df['DataVenda'] = pd.to_datetime(df['DataVenda'], format='%d/%m/%Y', errors='coerce')

In [31]:
pd.set_option("display.max_colwidth", 150)
pd.set_option("display.min_rows", 20)


Grava os dados do Dataframe no Banco de Dados

In [32]:
# Carrega para o PostgreSQL (substitui a tabela se já existir)
df.to_sql('TB_VENDAS_EMPRESA_B_GOLD', engine, if_exists='replace', index=False)


80

Utilizando script SQL para fazer o agrupamento e consolidar a base GOLD

In [3]:
import urllib

engine2 = get_sqlserver_engine()

df2 = pd.read_sql("""SELECT NomeItem, 
ClassificacaoProduto,  
SUBSTRING(DataVenda_Formatada, 1,10) AS "DataVenda", 
Estado AS "EstadoFilial",
SUM(CAST(ValorReais AS NUMERIC(18, 2))) AS "ValorTotalVenda"
FROM TB_VENDAS_EMPRESA_A_SILVER
GROUP BY NomeItem, 
ClassificacaoProduto,
DataVenda_Formatada,
Estado""", engine2)

df2['DataVenda'] = pd.to_datetime(df2['DataVenda'], format='%d/%m/%Y', errors='coerce')

df2['ValorTotalVenda'] = df2['ValorTotalVenda'].astype(str).str.replace(',', '.', regex=False)

print(df2.head())

       NomeItem ClassificacaoProduto  DataVenda       EstadoFilial  \
0  Base líquida            Alimentos 2023-07-01   Distrito Federal   
1  Base líquida            Alimentos 2023-04-02  Rio Grande do Sul   
2  Base líquida            Alimentos 2023-05-03     Espirito Santo   
3  Base líquida            Alimentos 2023-06-03              Ceara   
4  Base líquida            Alimentos 2022-12-05           Amazonas   

  ValorTotalVenda  
0           33.96  
1           64.88  
2           15.26  
3            27.9  
4           78.27  


Gravando os dados no SQL Server

In [47]:
# Envia os dados para o SQL Server
df2.to_sql("TB_VENDAS_EMPRESA_A_GOLD", engine2, if_exists="replace", index=False)


64

Disponibiliza os dados no diretorio local em CSV

In [4]:
caminho_arquivo = ('../data_processed/dados_vendas_empresa_a.csv')
df2.to_csv(caminho_arquivo, index=False, sep=';')